<a href="https://colab.research.google.com/github/diqnfl777/2022F-Ajou-ML-TEAM3/blob/main/LibraryModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 패키지 설치 및 드라이브 마운트

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# 트랜스포머 설치
!pip install transformers
from transformers import AutoTokenizer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master
import tensorflow as tf
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings

from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
from tqdm import tqdm, tqdm_notebook


''' 사용예시: 이런식으로 사용하여 TPU로 모델 학습 가능
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.TPUStrategy(resolver)
def create_model():
  return tf.keras.Sequential(
      [tf.keras.layers.Conv2D(256, 3, activation='relu', input_shape=(28, 28, 1)),
       tf.keras.layers.Conv2D(256, 3, activation='relu'),
       tf.keras.layers.Flatten(),
       tf.keras.layers.Dense(256, activation='relu'),
       tf.keras.layers.Dense(128, activation='relu'),
       tf.keras.layers.Dense(10)])

with strategy.scope():
  model = create_model()
  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['sparse_categorical_accuracy'])
'''

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-r2zhbn4q
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-r2zhbn4q


" 사용예시: 이런식으로 사용하여 TPU로 모델 학습 가능\nresolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])\ntf.config.experimental_connect_to_cluster(resolver)\ntf.tpu.experimental.initialize_tpu_system(resolver)\nstrategy = tf.distribute.TPUStrategy(resolver)\ndef create_model():\n  return tf.keras.Sequential(\n      [tf.keras.layers.Conv2D(256, 3, activation='relu', input_shape=(28, 28, 1)),\n       tf.keras.layers.Conv2D(256, 3, activation='relu'),\n       tf.keras.layers.Flatten(),\n       tf.keras.layers.Dense(256, activation='relu'),\n       tf.keras.layers.Dense(128, activation='relu'),\n       tf.keras.layers.Dense(10)])\n\nwith strategy.scope():\n  model = create_model()\n  model.compile(optimizer='adam',\n                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),\n                metrics=['sparse_categorical_accuracy'])\n"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#KoBERT 
#https://colab.research.google.com/github/SKTBrain/KoBERT/blob/master/scripts/NSMC/naver_review_classifications_pytorch_kobert.ipynb#scrollTo=jlj7KeoNV5yc
#위는 KoBERT 사용 네이버 리뷰 분석
from kobert import get_tokenizer
from kobert import get_pytorch_kobert_model
kr_tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
kr_model, kr_vocab = get_kobert_model('skt/kobert-base-v1',tokenizer.vocab_file)

## 데이터 가공 파트


In [41]:
#데이터 가공
#청구기호 숫자 앞 2개만 따와서 각각 매핑. 65~70은 예외적으로 경영학
data = pd.read_csv("/content/gdrive/MyDrive/LibraryCsv/hapbontocsv.CSV",encoding = 'cp949')
data = data.loc[:,['제목', '청구번호']]
data = data.dropna()
data

,제목,청구번호
0,海南 黃山理 分吐遺蹟.1,951.988 a
1,대학수학능력시험의 영어 독해문제 분석 및 개선방안,000 이66대
2,韓民族 統一聯邦國家 建設을 위한 올바른 歷史의 認識과 그 成就를 위한 提言 : 지...,000 이883한
3,The consultant's calling : bringing who you ar...,001 B445c
4,The flawless consulting fieldbook and companio...,001 B651f
...,...,...
436311,파라과이 便覽,R 989.2 대658파
436312,우루과이 便覽,R 989.5 대658우 `96
436313,뉴질랜드 便覽,R 993 대658뉴'96
436314,Year book Australia,R 994 C346y `86


In [42]:
import re 
data['청구번호'] = data['청구번호'].replace({r'(.*?)(\d{2})\d.*' : r'\2'}, regex=True)
data

,제목,청구번호
0,海南 黃山理 分吐遺蹟.1,95
1,대학수학능력시험의 영어 독해문제 분석 및 개선방안,00
2,韓民族 統一聯邦國家 建設을 위한 올바른 歷史의 認識과 그 成就를 위한 提言 : 지...,00
3,The consultant's calling : bringing who you ar...,00
4,The flawless consulting fieldbook and companio...,00
...,...,...
436311,파라과이 便覽,98
436312,우루과이 便覽,98
436313,뉴질랜드 便覽,99
436314,Year book Australia,99


In [43]:
data['청구번호'] = pd.to_numeric(data['청구번호'],errors = 'coerce')
data = data.dropna()
data.loc[(data['청구번호'] < 10), ['청구번호']] = 1000
data.loc[(data['청구번호'] < 20), ['청구번호']] = 1001
data.loc[(data['청구번호'] < 30), ['청구번호']] = 1002
data.loc[(data['청구번호'] < 40), ['청구번호']] = 1003
data.loc[(data['청구번호'] < 50), ['청구번호']] = 1004
data.loc[(data['청구번호'] < 60), ['청구번호']] = 1005
data.loc[(data['청구번호'] < 65), ['청구번호']] = 1006 #경영학쪽은 따로 분류이기 떄문에 65 사용
data.loc[(data['청구번호'] < 70), ['청구번호']] = 1007 #즉, 1007 쪽이 경영학책
data.loc[(data['청구번호'] < 80), ['청구번호']] = 1008
data.loc[(data['청구번호'] < 90), ['청구번호']] = 1009
data.loc[(data['청구번호'] <  100), ['청구번호']] = 1010

labels = {'0':'총류',
          '1':'철학',
          '2':'종교',
          '3':'사회학',
          '4':'언어',
          '5':'자연과학',
          '6':'기술과학',
          '7':'경영학',
          '8':'예술',
          '9':'문학',
          '10':'역사'
          }

In [44]:
data['청구번호'] = data['청구번호']%1000
data['청구번호'] = data['청구번호'].astype(int)


In [49]:
data['청구번호'] = data['청구번호'].apply(str) #스트링 변환
data['청구번호'] = data['청구번호'].replace(labels)
data

,제목,청구번호
0,海南 黃山理 分吐遺蹟.1,역사
1,대학수학능력시험의 영어 독해문제 분석 및 개선방안,총류
2,韓民族 統一聯邦國家 建設을 위한 올바른 歷史의 認識과 그 成就를 위한 提言 : 지...,총류
3,The consultant's calling : bringing who you ar...,총류
4,The flawless consulting fieldbook and companio...,총류
...,...,...
436311,파라과이 便覽,역사
436312,우루과이 便覽,역사
436313,뉴질랜드 便覽,역사
436314,Year book Australia,역사


In [45]:
#input_string = "Peace-building and development in Guatemala and Northern Ireland"
regex = '.*[ㄱ-ㅎ|ㅏ-ㅣ|가-힣].*'
kor_data = data[data.제목.str.match(regex)]                                 # 한글이 있다면 따로 빼내기
eng_data = pd.concat([data, kor_data, kor_data]).drop_duplicates(keep=False)

kor_data

eng_data = eng_data.sample(frac = 0.1)

###KOR


### ENG

In [46]:
from tokenizers import Tokenizer
from transformers import BertTokenizer, BertForSequenceClassification, BertConfig
from torch.optim import Adam

en_tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

class EN_Dataset(Dataset):
    def __init__(self, df):
        self.labels =  df['청구번호']
        self.texts = [en_tokenizer(text, padding='max_length', max_length = 512, truncation=True, return_tensors="pt") for text in df['제목']]
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        text = self.texts[idx]
        label = torch.tensor(self.labels[idx])
        return text, label

In [47]:
device = torch.device("cuda")
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased',num_labels = 11)
model.to(device)
train_dataset = EN_Dataset(eng_data)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [48]:
train_dataset.__getitem__(0)

KeyError: ignored

In [39]:
optimizer = Adam(model.parameters(), lr=1e-6)

itr = 1
p_itr = 500
epochs = 1
total_loss = 0
total_len = 0
total_correct = 0

model.train()
for epoch in range(epochs):
    
    for i, batch in enumerate(tqdm(train_loader)):
        text, label = batch
        optimizer.zero_grad()
        sample = text
        sample, label = sample.to(device), label.to(device)
        labels = torch.tensor(label)
        outputs = model(sample, labels=labels)
        loss, logits = outputs

        pred = torch.argmax(F.softmax(logits), dim=1)
        correct = pred.eq(labels)
        total_correct += correct.sum().item()
        total_len += len(labels)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        
        if itr % p_itr == 0:
            print('[Epoch {}/{}] Iteration {} -> Train Loss: {:.4f}, Accuracy: {:.3f}'.format(epoch+1, epochs, itr, total_loss/p_itr, total_correct/total_len))
            total_loss = 0
            total_len = 0
            total_correct = 0
        itr+=1

  0%|          | 0/1098 [00:00<?, ?it/s]


KeyError: ignored